In [1]:
import numpy as np
import tensorflow as tf

import argparse
import time
import os


# from LSTM import Model
# from LSTM import load_pretrained_model
import LSTM
from LSTM import *
from importlib import reload
reload(LSTM)
from dataloader import DataProcess
from sample import *
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt

In [ ]:
def init_args():
    
        args = {}
        args['rnn_size'] = 100 
        args['tsteps'] = 300 
        args['batch_size'] = 32 #32 #50 #32
        args['num_batches'] = 500 #32 
        args['num_mixtures'] = 20
        args['learning_rate'] = 0.005
        args['alphabet'] = ' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
        args['tsteps_per_ascii'] = 25
        args['epochs'] = 2500 #100
        args['biases'] = 1.0
        args['logs_dir'] = './logs/'
        args['save_path'] = 'saved2/model6/model.ckpt' #'saved2/model.ckpt-3000'
        args['load_path'] = 'saved2/model5/model.ckpt'
        args['sleep_time'] = 5 #60*5
        args['grad_clip'] = 10
        args['n_to_save'] = 500
        args['scale_factor'] = 20
        args['gap'] = 500
        args['learning_rate_decay'] = 0.95
        args['dropout_prob'] = 0.85
        args['train'] = False
        args['decay'] = 0.95
        args['momentum'] = 0.9
        return args

def load_pretrained_model(model, path):
        global_step = 0
        load_was_success = True
        try:
            save_dir = '/'.join(path.split('/')[:-1])
            ckpt = tf.train.get_checkpoint_state(save_dir)
            load_path = ckpt.model_checkpoint_path
            model.saver.restore(model.sess, load_path)
            #load_was_success = True
        except Exception as e:
#             print(e)
            load_was_success = False
        else:
            model.saver = tf.train.Saver(tf.global_variables())
            global_step = int(load_path.split('-')[-1])
            load_was_success = True
        return load_was_success, global_step
    
def train_model():
    sess = tf.Session()
    args = init_args()
    args['train'] = True
    data_loader = DataProcess(args)
    args['num_batches'] = data_loader.num_batches
    print("num_bacthes", args['num_batches'])
    model = Model(args)
    load_was_success, global_step = load_pretrained_model(model, args['save_path'])
#     global_step = 0
    v_x, v_y = data_loader.get_validation_data()
    valid_inputs = {model.input: v_x, model.output: v_y}
    plot_loss = []
    model.sess.run(tf.assign(model.decay, args['decay']))
    model.sess.run(tf.assign(model.momentum, args['momentum']))
    for e in range(int(global_step/args['num_batches']), args['epochs']):
        model.sess.run(tf.assign(model.learning_rate, args['learning_rate'] * (args['learning_rate_decay'] ** e)))
        print("Running epoch", e)
        data_loader.init_batch_comp()
        c0, c1, c2 = model.istate_cell0.c.eval(), model.istate_cell1.c.eval(), model.istate_cell2.c.eval()
        h0, h1, h2 = model.istate_cell0.h.eval(), model.istate_cell1.h.eval(), model.istate_cell2.h.eval()

        for b in range(global_step%args['num_batches'], args['num_batches']):
            print(b)

            i = e * args['num_batches'] + b
            if global_step is not 0 : i+=1 ; global_step = 0

            if i % args['n_to_save'] == 0 and (i > 0):
                model.saver.save(model.sess, args['save_path'], global_step = i) ;

            start = time.time()
            x, y = data_loader.get_next_batch()
            
            feed = {model.input: x, model.output: y, \
                    model.istate_cell0.c: c0, model.istate_cell1.c: c1, model.istate_cell2.c: c2, \
                    model.istate_cell0.h: h0, model.istate_cell1.h: h1, model.istate_cell2.h: h2}
            [train_loss, _] = model.sess.run([model.cost, model.train_op], feed)

            end = time.time()
            print("train_loss: " + str(i))
            print(train_loss)
            plot_loss.append(train_loss)
#             if i % 10 is 0:
#                 print("train_loss: " + str(i))
#                 print(train_loss)
#                 plot_loss.append(train_loss)
    plt.plot(plot_loss, linewidth=2.0)
    plt.savefig("./loss.png")

def sample_model():
    args = init_args()
    args['tsteps'] = 1
    args['batch_size'] = 1
#     if args['text'] == '':
#     strings = ['Machine generated handwriting'] # test strings
#     else:
#         strings = [args['text']]

    model = Model(args)

    load_was_success, global_step = load_pretrained_model(model, args['load_path'])
    print(global_step)
    s = "hello world"
    if load_was_success:
            strokes, char_to_plot = sample(model, args)
            l_save_path = '{}figures/iter-{}-l-{}'.format(args['logs_dir'], global_step, s[:10].replace(' ', '_'))

            gauss_plot(strokes, 'Heatmap for "{}"'.format(s), figsize = (2*len(s),4), save_path="./gauss5.png")
#             line_plot(strokes, 'Line plot for "{}"'.format(s), figsize = (len(s),2), save_path="./line5.png")
            line_plot(strokes, 'Line plot for "{}"'.format(s), figsize = (len(s),2), save_path=l_save_path)
            line_plot3(strokes, char_to_plot, 'Line plot for "{}"'.format(s), figsize = (len(s),2), save_path="./line_char5.png")
#             line_plot2(char_to_plot, 'Line plot for "{}"'.format(s), figsize = (len(s),2), save_path="./line_char6.png")
            print("plotted")

    else:
        print("load failed, sampling canceled")

#     if True:
#         print("calling sample again")
#         tf.reset_default_graph()
#         time.sleep(args['sleep_time'])
#         sample_model()
        
# def sample_new():
#     i = 0
#     while(i < 5):
#         sample_model()
#         i++

            
            
            
            
            
            
            
            
            
    

In [ ]:
train_model()

Saved {} lines 226
Number of data examples: 202
Batch size for dataset 6
num_bacthes 6

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from saved2/model6/model.ckpt-1500
Running epoch 250
0
